<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/All_in_One_Indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 4.9 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=e1289b81e580ccfea059237130831319ed9bb011b859d3e56b83af96a1408be6
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 128 kB 9.4 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 110 kB 40.2 MB/s 
     |████████████████████████████████| 112 kB 39.5 MB/s 
     |████████████████████████████████| 4.0 MB 42.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling request

In [7]:
import yfinance as yfin
yfin.pdr_override()
import pandas as pd
import pandas_ta as ta

import numpy as np
import plotly.graph_objs as go
import plotly
import plotly.subplots

def get_all_in_one_indicators(stock):
  df = yfin.Ticker(stock).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # calculate MACD values
  df['200_ema'] = round(df['close'].ewm(span = 200, adjust = False).mean(),0) 
  df['50_ema'] = round(df['close'].ewm(span = 50, adjust = False).mean(),0) 
  df.ta.macd(close='close', fast=12, slow=26, append=True)
  df.ta.eom(append=True)
  df.ta.rsi(append=True)

  df = df[df.index > '2022-03-31']
  # Force lowercase (optional)
  df.columns = [x.lower() for x in df.columns]
  # Construct a 2 x 1 Plotly figure
  fig = plotly.subplots.make_subplots(rows=4, cols=1)
  # price Line
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['close'],
          line=dict(color='#0000ff', width=1),
          name='price',
          # showlegend=False,
          # legendgroup='1',
      ), row=1, col=1
  )
  # EMA 200
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['200_ema'],
          line=dict(color='#000000', width=2),
          name='200 ema',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )
  # EMA 50
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['50_ema'],
          line=dict(color='#ff9900', width=2),
          name='50 ema',
          # showlegend=False,
          # legendgroup='2',
      ), row=1, col=1
  )
  # Fast Signal (%k)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macd_12_26_9'],
          line=dict(color='#ff9900', width=2),
          name='macd',
          # showlegend=False,
          # legendgroup='2',
      ), row=2, col=1
  )
  # Slow signal (%d)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['macds_12_26_9'],
          line=dict(color='#000000', width=2),
          # showlegend=False,
          # legendgroup='2',
          name='signal'
      ), row=2, col=1
  )
  # Colorize the histogram values
  colors = np.where(df['macdh_12_26_9'] < 0, '#000', '#ff9900')
  # Plot the histogram
  fig.append_trace(
      go.Bar(
          x=df.index,
          y=df['macdh_12_26_9'],
          name='histogram',
          marker_color=colors,
      ), row=2, col=1
  )
  # RSI
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['rsi_14'],
          line=dict(color='#00ff00', width=2),
          name='rsi',
          # showlegend=False,
          legendgroup='2',
      ), row=3, col=1
  )
  # EOM
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['eom_14_100000000'],
          line=dict(color='#ff0000', width=2),
          name='eom',
          # showlegend=False,
          legendgroup='2',
      ), row=4, col=1
  )
  # Make it pretty
  layout = go.Layout(
      plot_bgcolor='#efefef',
      # Font Families
      font_family='Monospace',
      font_color='#000000',
      font_size=12,
      width=1500, 
      height=1000,
      xaxis=dict(
          rangeslider=dict(
              visible=False
          )
      )
  )
  # Update options and show plot
  fig.update_layout(layout)
  fig.show()

stock = 'JUBLFOOD.NS'
get_all_in_one_indicators(stock)